In [221]:
import torch

In [222]:
X = torch.randn(2,3,4)
X = torch.abs(X)

snažím se teď naimplementovat exclusive cumsum

In [223]:
dim = -1
rolled = torch.roll(X,1,dims=dim)
rolled

tensor([[[1.2654, 1.8297, 1.2963, 1.1808],
         [0.6911, 0.4490, 1.0687, 1.3556],
         [0.7595, 0.3680, 1.2219, 1.1132]],

        [[1.2596, 0.0498, 0.1093, 0.2504],
         [1.0404, 0.6790, 1.6915, 0.4595],
         [0.1868, 0.0625, 0.4495, 1.9826]]])

Vynulovat první index, podel dané dimenze
1. trik je ve využití slice(None), jelikož v tomhle případě zanmená :

In [224]:
select = [slice(None)] * rolled.dim()
select[dim] = 0

In [225]:
rolled[select] = 0
rolled

tensor([[[0.0000, 1.8297, 1.2963, 1.1808],
         [0.0000, 0.4490, 1.0687, 1.3556],
         [0.0000, 0.3680, 1.2219, 1.1132]],

        [[0.0000, 0.0498, 0.1093, 0.2504],
         [0.0000, 0.6790, 1.6915, 0.4595],
         [0.0000, 0.0625, 0.4495, 1.9826]]])

In [226]:
torch.cumsum(rolled,dim=dim)

tensor([[[0.0000, 1.8297, 3.1261, 4.3069],
         [0.0000, 0.4490, 1.5177, 2.8733],
         [0.0000, 0.3680, 1.5899, 2.7031]],

        [[0.0000, 0.0498, 0.1591, 0.4095],
         [0.0000, 0.6790, 2.3705, 2.8300],
         [0.0000, 0.0625, 0.5120, 2.4946]]])

In [227]:
def exclusive_cumsum(x: torch.Tensor, dim: int):
    rolled = torch.roll(x, 1, dims=dim)
    select = [slice(None)] * x.dim()
    select[dim] = 0
    rolled[select] = 0
    return torch.cumsum(rolled, dim=dim)

In [228]:
X

tensor([[[1.8297, 1.2963, 1.1808, 1.2654],
         [0.4490, 1.0687, 1.3556, 0.6911],
         [0.3680, 1.2219, 1.1132, 0.7595]],

        [[0.0498, 0.1093, 0.2504, 1.2596],
         [0.6790, 1.6915, 0.4595, 1.0404],
         [0.0625, 0.4495, 1.9826, 0.1868]]])

In [229]:
sorted, indices = torch.sort(X, dim=dim, descending=True)
sorted,indices

(tensor([[[1.8297, 1.2963, 1.2654, 1.1808],
          [1.3556, 1.0687, 0.6911, 0.4490],
          [1.2219, 1.1132, 0.7595, 0.3680]],
 
         [[1.2596, 0.2504, 0.1093, 0.0498],
          [1.6915, 1.0404, 0.6790, 0.4595],
          [1.9826, 0.4495, 0.1868, 0.0625]]]),
 tensor([[[0, 1, 3, 2],
          [2, 1, 3, 0],
          [1, 2, 3, 0]],
 
         [[3, 2, 1, 0],
          [1, 3, 0, 2],
          [2, 1, 3, 0]]]))

In [230]:
cdf = exclusive_cumsum(sorted,dim)
cdf

tensor([[[0.0000, 1.8297, 3.1261, 4.3915],
         [0.0000, 1.3556, 2.4243, 3.1154],
         [0.0000, 1.2219, 2.3351, 3.0946]],

        [[0.0000, 1.2596, 1.5100, 1.6193],
         [0.0000, 1.6915, 2.7318, 3.4109],
         [0.0000, 1.9826, 2.4321, 2.6189]]])

In [231]:
mask = cdf <= 1.5
mask

tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[ True,  True, False, False],
         [ True, False, False, False],
         [ True, False, False, False]]])

Verze s scatterem

Gather vytvoří nový tensor z původního, kde index tensor uručuje index v dané dimenzi.
* out[i][j][k] = src[i][j][index[i][j][k]]

Scatter by měla být udajně opačná operace vůči gather
* out[i][j][index[i][j][k]] = src[i][j][k]

mask_full

In [232]:
X

tensor([[[1.8297, 1.2963, 1.1808, 1.2654],
         [0.4490, 1.0687, 1.3556, 0.6911],
         [0.3680, 1.2219, 1.1132, 0.7595]],

        [[0.0498, 0.1093, 0.2504, 1.2596],
         [0.6790, 1.6915, 0.4595, 1.0404],
         [0.0625, 0.4495, 1.9826, 0.1868]]])

In [233]:
full_mask = torch.zeros_like(indices,dtype=torch.bool)
full_mask.scatter_(dim=dim,index=indices,src=cdf > 1.5)
torch.masked_fill(X,full_mask,0)

tensor([[[1.8297, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0687, 1.3556, 0.0000],
         [0.0000, 1.2219, 1.1132, 0.0000]],

        [[0.0000, 0.0000, 0.2504, 1.2596],
         [0.0000, 1.6915, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.9826, 0.0000]]])

Teď asi potřebuju zkonvertovat, každý prvek na index, takže je ptřeba vytvořit mapu každého prvku

* vzít shape[-1] 
* zavolat arange -> cumprod(shape[:-1])
* reshape -> shape

In [234]:
shape_t = torch.tensor(indices.shape)
addition =  torch.reshape(torch.arange(start=0,end=torch.cumprod(shape_t,dim=0)[-1].item(),step=indices.shape[-1]),indices.shape[:-1]).unsqueeze(-1)
addition

tensor([[[ 0],
         [ 4],
         [ 8]],

        [[12],
         [16],
         [20]]])

In [235]:
indices

tensor([[[0, 1, 3, 2],
         [2, 1, 3, 0],
         [1, 2, 3, 0]],

        [[3, 2, 1, 0],
         [1, 3, 0, 2],
         [2, 1, 3, 0]]])

In [236]:
indices = indices + addition
indices

tensor([[[ 0,  1,  3,  2],
         [ 6,  5,  7,  4],
         [ 9, 10, 11,  8]],

        [[15, 14, 13, 12],
         [17, 19, 16, 18],
         [22, 21, 23, 20]]])

In [237]:
indices[mask] = -1
indices

tensor([[[-1,  1,  3,  2],
         [-1, -1,  7,  4],
         [-1, -1, 11,  8]],

        [[-1, -1, 13, 12],
         [-1, 19, 16, 18],
         [-1, 21, 23, 20]]])

In [238]:
X[torch.unravel_index(indices[indices > 0],indices.shape)] = 0
X

tensor([[[1.8297, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0687, 1.3556, 0.0000],
         [0.0000, 1.2219, 1.1132, 0.0000]],

        [[0.0000, 0.0000, 0.2504, 1.2596],
         [0.0000, 1.6915, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.9826, 0.0000]]])